In [0]:
%load_ext autoreload
%autoreload 2

import matplotlib
from matplotlib import pyplot
pyplot.ion()
pyplot.style.use('dark_background')

import torch as th
from rl.core.algs.dqn import *

In [2]:
from rl.core.algs import experiment
from rl.core.algs import model
from rl.core.envs import grid_world

BATCH_SIZE = 32
TRAIN_START_STEP = 3 * BATCH_SIZE
NUM_TRAIN_STEPS = 10000 # 17
REPLAY_SIZE = 100000
TARGET_NETWORK_UPDATE_FREQ = 500


env = environment.GridWorldWrapped(grid_shape=(4, 4, 1))
conv_specs = {
  model.ConvSpec(32, 4, 1),
}
fc_specs = (64,)
model_params = {
  'conv_specs': conv_specs,
  'fc_specs': fc_specs,
}

EPS_DECAY = util.Schedule((0, TRAIN_START_STEP, NUM_TRAIN_STEPS / 1, NUM_TRAIN_STEPS),
                          (1, 1, 0.0, 0.0))
EPS_RANDOM = util.Schedule((0,), (1.0,))
adam_params = {
  'fn': th.optim.Adam,
  'kwargs': {
    'lr': 0.005,
  }
}
rmsprop_params = {
  'fn': th.optim.RMSprop,
  'kwargs': {
    'lr': 0.00025,
    'momentum': 0.95,
    'alpha': 0.95,
    'eps': 0.01,
  }
}

policy_params = {
    'opt_params': rmsprop_params,
    'eps_sched': EPS_DECAY,  # TODO: change
    'target_update_freq': TARGET_NETWORK_UPDATE_FREQ,
}
exp = experiment.Experiment(env, model.QNetwork, model_params,
                            Dqn, policy_params)

print(exp.policy.model)

rb = ReplayBuffer(REPLAY_SIZE)
for step in range(TRAIN_START_STEP):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)

eps_rew = 0
for step in range(TRAIN_START_STEP, NUM_TRAIN_STEPS):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)
    
  experience_batch = rb.sample(BATCH_SIZE)
  m = exp.policy.update(experience_batch)
  
  eps_rew += sars.r
  if sars.s1 is None:
    m['r_per_eps'] = eps_rew
    eps_rew = 0
  if step % 100 == 0 or sars.s1 is None:
    for k, v in m.items():
      exp.plt.add_data(k, step, v)
      
# Expect about r_per_eps=-50 in pure random agent.

(1, 1, 32)
QNetwork(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [3]:
opt = grid_world.GridWorld((4, 4)).optimal_expected_reward()
print(f'optimal expected r: {opt}')

optimal expected r: 1.6666666666666665


In [4]:
env.show_model_policy(exp.policy.model)

[[ 0.         -0.03565243 -0.98968899 -1.94701207]
 [ 0.09208554 -0.94079018 -1.91302907 -2.97501612]
 [-0.96546632 -1.92566752 -2.88638926 -3.92972302]
 [-1.99731803 -3.00002432 -3.97250724 -4.70307589]]
[['⟲' '←' '←' '←']
 ['↑' '↑' '←' '←']
 ['↑' '↑' '↑' '↑']
 ['↑' '↑' '←' '↑']]
[[ 0.06983802  0.         -0.12369719 -1.0901283 ]
 [-0.92832971 -0.07376561 -1.05427289 -2.08433652]
 [-2.02989602 -1.02859521 -2.07932711 -3.0136981 ]
 [-2.99683666 -2.02935934 -3.03669262 -3.65212059]]
[['→' '⟲' '←' '←']
 ['→' '↑' '↑' '←']
 ['↑' '↑' '←' '↑']
 ['↑' '↑' '↑' '↑']]
[[-0.98799074 -0.17828527  0.         -0.00574729]
 [-1.89138412 -0.97470677  0.08617634 -0.95501506]
 [-2.89577818 -1.92607808 -0.90306997 -2.0723877 ]
 [-3.78357339 -2.87495208 -2.05026913 -2.85262084]]
[['→' '→' '⟲' '←']
 ['↑' '↑' '↑' '←']
 ['↑' '↑' '↑' '↑']
 ['↑' '↑' '↑' '↑']]
[[-1.90956759e+00 -1.08513665e+00 -1.04762822e-01  0.00000000e+00]
 [-2.87019324e+00 -1.88199759e+00 -1.02859175e+00 -1.06751919e-03]
 [-3.46420312e+00 -2

In [5]:
# Debug the action-value function for a given state.
s = [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 2, 0]]
s = np.expand_dims(np.expand_dims(s, 0), -1)
print(exp.policy.model(util.to_tensor(s)))
print(exp.policy.target_model(util.to_tensor(s)))

tensor([[-2.0365, -2.1169, -0.0119, -1.9946]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([[-1.9510, -2.0999,  0.0151, -1.9945]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)


In [6]:
# Examine the replay buffer.
terminals = rb.filter_by(sars_filter(r=0))
print(len(terminals))

# action-value at 's' was wrong. Search the replay buffer for occurrences of
# the correct transition. It's likely missing, which led the NN to overfit
# on missing data.
trans_s = rb.filter_by(sars_filter(s=s))
trans_s_a = rb.filter_by(sars_filter(s=s,a=2))
print(f'Num with (s): {len(trans_s)}. Num with (s, a): {len(trans_s_a)}')


1673
Num with (s): 75. Num with (s, a): 56


In [7]:
env.visualize(exp.policy, steps=20)

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 2]])
array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 2]])
array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 2]])
array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1]])
array([[0, 0, 0, 2],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 1, 0, 2],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 1, 2],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 1, 2],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [1, 2, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])
array([[0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
  